# 골빈해커 3분딥러닝 텐서플로맛 Ch4
## 기본 신경망 구현

## 간단한 분류 모델

### Task : 털, 날개가 있는지를 기준으로 포유류와 조류를 구분하는 신경망 모델

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\Ha-yeon\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


학습에 사용할 데이터 정의. <br>
[털, 날개]

In [2]:
x_data = np.array([[0,0], [1,0], [1,1], [0,0], [0,0], [0,1]])

[기타, 포유류, 조류] one-hot encoding

In [4]:
y_data = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

신경망 모델 구성. <br>
특징 X와 레이블 Y 와의 관계를 알아내는 모델을 짤 것인데, X와 Y에 실측값(ground truth)를 넣어서 학습시킬 것이므로 X와 Y는 플레이스 홀더로 설정

In [5]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

가중치와 편향 변수를 변경

In [6]:
W = tf.Variable(tf.random_uniform([2,3], -1., 1))
b = tf.Variable(tf.zeros([3]))

In [7]:
L = tf.add(tf.matmul(X,W), b)
L = tf.nn.relu(L)

이게 신경망 구성 끝이다. 여기까진 Weight를 곱하고 bias를 더한 결과를 activation function(relu)에 적용한 것. <br> 
뒤에는 Loss계산이 남아있음. <br>
일단 신경망을 통해 나온 출력값을 softmax를 써서 사용하기 쉽게 다듬는다.

In [10]:
model = tf.nn.softmax(L)

손실함수. Cross Entropy 사용.

In [11]:
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(model), axis=1))

학습

In [17]:
# optimize with gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(cost)

In [22]:
# 텐서플로우세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞에서 구성한 특징과 레이블을 이용해 학습을 100번 진행
for step in range(100):
    sess.run(train_op,feed_dict = {X:x_data, Y:y_data})
    
    #학습 도중 10번에 한번씩 손실값 출력
    if (step + 1)%10 ==0:
        print(step + 1, sess.run(cost, feed_dict={X:x_data, Y:y_data} ))

10 1.1410152
20 1.138394
30 1.1357986
40 1.1332358
50 1.1307055
60 1.1282061
70 1.1257372
80 1.1232977
90 1.1208565
100 1.1184493


학습 결과를 보는 코드

In [23]:
prediction = tf.argmax(model, axis = 1)
target = tf.argmax(Y, axis = 1)
print('예측값 :', sess.run(prediction, feed_dict = {X:x_data}))
print('실제값 :', sess.run(target, feed_dict = {Y:y_data}))

예측값 : [0 2 2 0 0 0]
실제값 : [0 1 2 0 0 2]


정확도 Accuracy

In [24]:
is_correct = tf.equal(prediction,target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict = {X:x_data, Y:y_data}))

정확도: 66.67


## 4.3 심층 신경망 구현


In [30]:
import tensorflow as tf
import numpy as np

x_data = np.array([[0,0], [1,0], [1,1], [0,0], [0,0], [0,1]])
y_data = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)



두 층 이상의 심층 신경망 구현.

앞에서 만든 신경망 모델에 weight와 bias만 추가하면 된다.

In [31]:
W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10,3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([13]))

특징 입력값에 첫번째 weight와 bias, 그리고 activation fn을 적용

In [32]:
L1 = tf.add(tf.matmul(X,W1), b1)
L1 = tf.nn.relu(L1)

출력층을 만들기 위해 두번째 weight와 bias를 적용. hidden layer에 W2와 b2를 적용하면 3개의출력값을 가진다. 

In [33]:
model = tf.add(tf.matmul(L1,W2), b2)


ValueError: Dimensions must be equal, but are 3 and 13 for 'Add_4' (op: 'Add') with input shapes: [?,3], [13].

In [36]:

import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

# [기타, 포유류, 조류]
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

#########
# 신경망 모델 구성
######
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 첫번째 가중치의 차원은 [특성, 히든 레이어의 뉴런갯수] -> [2, 10] 으로 정합니다.
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
# 두번째 가중치의 차원을 [첫번째 히든 레이어의 뉴런 갯수, 분류 갯수] -> [10, 3] 으로 정합니다.
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

# 편향을 각각 각 레이어의 아웃풋 갯수로 설정합니다.
# b1 은 히든 레이어의 뉴런 갯수로, b2 는 최종 결과값 즉, 분류 갯수인 3으로 설정합니다.
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

# 신경망의 히든 레이어에 가중치 W1과 편향 b1을 적용합니다
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

# 최종적인 아웃풋을 계산합니다.
# 히든레이어에 두번째 가중치 W2와 편향 b2를 적용하여 3개의 출력값을 만들어냅니다.
model = tf.add(tf.matmul(L1, W2), b2)

# 텐서플로우에서 기본적으로 제공되는 크로스 엔트로피 함수를 이용해
# 복잡한 수식을 사용하지 않고도 최적화를 위한 비용 함수를 다음처럼 간단하게 적용할 수 있습니다.
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)


#########
# 신경망 모델 학습
######
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))


#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


10 0.8647298
20 0.6790374
30 0.5350161
40 0.4139134
50 0.31623447
60 0.2394215
70 0.17928094
80 0.13289528
90 0.09829834
100 0.07322309
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
